<a href="https://colab.research.google.com/github/Nikoschenk/higgs-audio/blob/main/clone_and_speak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install soundfile tqdm

# Step 1: Clone and change into repo directory & install requirements.
!git clone https://github.com/Nikoschenk/higgs-audio.git
%cd higgs-audio
!pip install huggingface_hub
from huggingface_hub import snapshot_download
snapshot_download(repo_id="bosonai/higgs-audio-v2-generation-3B-base", local_dir="model")

!pip install transformers==4.41.2 loguru click pyyaml
!apt-get install -y libsndfile1
!pip install "numpy<2.0" --upgrade --quiet
!pip install "dacite" --quiet
!pip install vector-quantize-pytorch --quiet
# ✅ Install from GitHub (not PyPI!)
!pip install git+https://github.com/descriptinc/audiotools.git --quiet

import os
print(os.listdir())
!touch boson_multimodal/__init__.py
!touch boson_multimodal/audio_processing/__init__.py
!touch boson_multimodal/audio_processing/descriptaudiocodec/__init__.py
!touch boson_multimodal/audio_processing/descriptaudiocodec/dac/__init__.py


# Clone Descript Audio Codec repo into the correct subdir
!git clone https://github.com/descriptinc/descript-audio-codec.git boson_multimodal/audio_processing/descriptaudiocodec/dac

# Fix missing __init__.py files
!touch boson_multimodal/__init__.py
!touch boson_multimodal/audio_processing/__init__.py
!touch boson_multimodal/audio_processing/descriptaudiocodec/__init__.py
!touch boson_multimodal/audio_processing/descriptaudiocodec/dac/__init__.py

# Install dependencies
!pip install -r requirements.txt
!pip install -e .
!pip install pyctcdecode phonemizer g2p-en jieba langid dacite vector-quantize-pytorch

# Install `audiotools` (Descript's)
!pip install git+https://github.com/descriptinc/audiotools.git

# Reinstall compatible versions (PyTorch 2.2.2 with torchvision 0.17.2)
!pip install -q --index-url https://download.pytorch.org/whl/cu121 torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1

# Remove torchao which is NOT required by your project but causes errors
!pip uninstall -y torchao

In [ ]:
import os
print(os.listdir())
from google.colab import files
# You ned to upload 3 files.
# 1) An audio recording, e.g., 1 minute, converted to .wav, e.g., "ni_de.wav".
# 2) A corresponding file containing the transcription of 1), e.g., "ni_de.txt".
# 3) A text file of what you want the TTS engine to speak, e.g., "de_test_ni.txt".
uploaded = files.upload()
# Show uploaded files.
print(os.listdir())

In [ ]:
import shutil
print(os.listdir())
# Place the audio files into the correct directories.
shutil.move("ni_de.txt", "examples/voice_prompts/ni_de.txt")
shutil.move("ni_de.wav", "examples/voice_prompts/ni_de.wav")
shutil.move("de_test_ni.txt", "examples/transcript/single_speaker/de_test_ni.txt")
print(os.listdir())

In [ ]:
%cd /content/higgs-audio

# Remove TF/JAX so Transformers won't import TensorFlow paths at all
!pip uninstall -y -q tensorflow tensorflow-cpu tensorflow-text tensorflow-io tensorflow-intel tf-nightly keras jax jaxlib || true

# (Optional) quick sanity check that TF is gone
import importlib.util
print("tensorflow present?", importlib.util.find_spec("tensorflow") is not None)

# Run with TF disabled (belt-and-suspenders)
!TRANSFORMERS_NO_TF=1 TF_CPP_MIN_LOG_LEVEL=3 HF_HUB_DISABLE_SYMLINKS_WARNING=1 \
python examples/generation.py \
  --model_path model \
  --audio_tokenizer bosonai/higgs-audio-v2-tokenizer \
  --transcript examples/transcript/single_speaker/de_test_ni.txt \
  --ref_audio ni_de \
  --seed 7788 \
  --out_path /content/output.wav \
  --device auto \
  --use_static_kv_cache 0


In [5]:
# Listen to the audio.
from IPython.display import Audio
Audio("/content/output.wav", rate=24000)
# Save the audio.
from google.colab import files
files.download("/content/output.wav")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>